In [2]:
import domolibrary.client.get_data as gd
import domolibrary.client.DomoError as dmde

In [24]:
import domolibrary.client.DomoAuth as dmda
import os

auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_ACCESS_TOKEN"],
    domo_instance=os.environ["DOMO_INSTANCE"],
)

await auth.who_am_i()

model_id = "48707704-213c-4c82-8a7d-69505b50a8de"
version_id = "1.0.4"
starting_block = "Start%20HelpDeskMafia"

In [4]:
async def get_workflow(
    auth: dmda.DomoAuth, model_id, version_id, debug_api: bool = False
):

    url = f"https://{auth.domo_instance}.domo.com/api/workflow/v1/models/{model_id}/versions/{version_id}"
    res = await gd.get_data(auth=auth, method="GET", url=url, debug_api=debug_api)

    if not res.is_success:
        raise dmde.RouteError(res=res)

    return res

In [5]:
res = await get_workflow(
    auth=auth, model_id=model_id, version_id=version_id, debug_api=False
)

from pprint import pprint

pprint(res.response["savedJson"])

('{"version":2,"designElements":[{"id":"rootNode","type":"rootNode","position":{"x":60,"y":30},"data":{"dimensions":{"width":200,"height":60},"title":"Start '
 'HelpDeskMafia","description":"","type":"Start","_designNode":"rootNode","isFormStart":false,"formId":null,"input":[{"children":[],"configType":null,"customMappingType":null,"dataType":"text","displayName":"question","entitySubType":null,"flag":"input","id":"nvdXKDGoXWJKIln","isList":false,"mappedTo":"HHwmUJbOuALMwWw","paramName":"question","required":true,"value":null,"visible":true,"engineName":"question"}]},"style":{"zIndex":3,"outline":"none"},"index":0},{"id":"zXTsAKikLbRiJaK","type":"AI_AGENT","position":{"x":60,"y":210},"data":{"dimensions":{"width":200,"height":60},"title":"FindKB '
 'Article","description":"This agent takes an input question and attempts to '
 'locate the most likely KB article and '
 'answer.","prompt":{"children":[],"configType":null,"customMappingType":null,"dataType":"text","displayName":"Prompt","e

In [6]:
import domolibrary.client.get_data as gd
import domolibrary.client.DomoError as dmde


def generate_trigger_workflow_body(starting_tile, model_id, version_id, execution_params : dict):
    return {
        "messageName": starting_tile,
        "version": version_id,
        "modelId": model_id,
        "data": execution_params,
    }


async def trigger_workflow(
    auth: dmda.DomoAuth,
    starting_tile: str,
    model_id: str,
    version_id: str,
    execution_parameters: dict = None,
    debug_api: bool = False,
):
    body = generate_trigger_workflow_body(
        starting_tile=starting_tile,
        model_id=model_id,
        execution_params=execution_parameters,
        version_id=version_id
    )

    url = f"https://{auth.domo_instance}.domo.com/api/workflow/v1/instances/message"

    res = await gd.get_data(
        method="POST", url=url, body=body, auth=auth, debug_api=debug_api
    )

    if not res.is_success:
        raise dmde.RouteError(res=res)

    return res

In [7]:
execution_parameters = {"question": "what is a beast mode"}
starting_tile = "Start HelpDeskMafia"

res = await trigger_workflow(
    auth=auth,
    starting_tile=starting_tile,
    model_id=model_id,
    version_id=version_id,
    execution_parameters= execution_parameters,
    debug_api=False,
)



In [8]:
res.response

{'id': '164b53d1-c9e3-48e3-828e-bf8971e988a2',
 'modelId': '48707704-213c-4c82-8a7d-69505b50a8de',
 'deploymentId': 'abc3bc1b-478a-49aa-8b8c-34d283a6f6df',
 'modelName': 'HelpDeskMafia',
 'modelVersion': '1.0.3',
 'bpmnProcessId': 'Process_48707704-213c-4c82-8a7d-69505b50a8de_1.0.3_1',
 'bpmnProcessName': 'mmmm-0012-0200 HelpDeskMafia 1.0.3 1',
 'createdBy': '475856860',
 'createdOn': '2025-03-16T21:39:31.127Z',
 'updatedBy': '475856860',
 'updatedOn': '2025-03-16T21:39:31.127Z',
 'status': 'IN_PROGRESS',
 'isTestRun': False}

In [9]:
async def get_workflow_trigger_history(auth : dmda.DomoAuth, model_id, debug_api : bool = False):

    url = f'https://{auth.domo_instance}.domo.com/api/workflow/v2/executions/{model_id}'

    res = await gd.get_data(auth=auth, method="GET", url=url, debug_api=debug_api)

    if not res.is_success:
        raise dmde.RouteError(res=res)
    
    return res


In [12]:
try:
    await get_workflow_trigger_history(auth = auth, model_id= model_id, debug_api= True)

except Exception as e:
    pass

🐛 debugging get_data
{'body': None,
 'function_name': 'get_traceback',
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': 'a9f28c9a3f100348ba97858f6f944ce97be421eb5817fcc7'},
 'method': 'GET',
 'params': None,
 'parent_class': None,
 'url': 'https://domo-community.domo.com/api/workflow/v2/executions/48707704-213c-4c82-8a7d-69505b50a8de'}
get_data: no body
get_data_response <Response [404 Not Found]>


In [21]:
async def get_workflow_executions(auth : dmda.DomoAuth,
                                  model_id: str,
                                  version_id: str,
                                  debug_api : bool = False
                                  ):
    params = {"modelId" : model_id,
            #   "triggerTypes" : "ALERT,API,APP_STUDIO,CUSTOM_APP,MANUAL,TIMER,WORKFLOW"
              "version" : version_id,
              # "status"  : "IN_PROGRESS"
    }
    
    url = f"https://{auth.domo_instance}.domo.com/api/workflow/v2/executions"

    res = await gd.get_data(auth=auth, method="GET", url=url, debug_api=debug_api, params = params)

    if not res.is_success:
        raise dmde.RouteError(res=res)
    
    return res

    

In [26]:
await get_workflow_executions(auth = auth,
                                  model_id = model_id,
                                  version_id = version_id,
                                  debug_api = True
)

res.response

🐛 debugging get_data
{'body': None,
 'function_name': 'get_traceback',
 'headers': {'Connection': 'keep-alive',
             'Content-Type': 'application/json',
             'accept': 'application/json, text/plain',
             'x-domo-developer-token': 'a9f28c9a3f100348ba97858f6f944ce97be421eb5817fcc7'},
 'method': 'GET',
 'params': {'modelId': '48707704-213c-4c82-8a7d-69505b50a8de',
            'version': '1.0.4'},
 'parent_class': None,
 'url': 'https://domo-community.domo.com/api/workflow/v2/executions'}
get_data: no body
get_data_response <Response [200 OK]>


{'id': '164b53d1-c9e3-48e3-828e-bf8971e988a2',
 'modelId': '48707704-213c-4c82-8a7d-69505b50a8de',
 'deploymentId': 'abc3bc1b-478a-49aa-8b8c-34d283a6f6df',
 'modelName': 'HelpDeskMafia',
 'modelVersion': '1.0.3',
 'bpmnProcessId': 'Process_48707704-213c-4c82-8a7d-69505b50a8de_1.0.3_1',
 'bpmnProcessName': 'mmmm-0012-0200 HelpDeskMafia 1.0.3 1',
 'createdBy': '475856860',
 'createdOn': '2025-03-16T21:39:31.127Z',
 'updatedBy': '475856860',
 'updatedOn': '2025-03-16T21:39:31.127Z',
 'status': 'IN_PROGRESS',
 'isTestRun': False}